In [ ]:
import cv2
import numpy as np
import sys
import os
methods = [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED, cv2.TM_CCORR, cv2.TM_CCORR_NORMED, cv2.TM_CCOEFF, cv2.TM_CCOEFF_NORMED]
DEBUG = "--debug" in sys.argv
SHOW_IMAGES = "--show" in sys.argv

def darken_yellow_outline(image):
    """
    Replace yellow pixels in an image with black ones. 
    Primarily used to remove the yellow outline on the puzzle piece in this captcha.
    :param image: Input image.
    :return: Output image with yellow pixels turned black.
    """
    # Convert image to BGR color space (with alpha channel)
    bgr = cv2.cvtColor(image, cv2.COLOR_RGBA2BGRA)
    if DEBUG:
        cv2.imwrite(f"debug_bgr_{captcha_id}.png", bgr)

    # Convert the BGR image to HSV image
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    if DEBUG:
        cv2.imwrite(f"debug_hsv_{captcha_id}.png", hsv)

    # Debug
    if DEBUG:
        hsv_pixel = hsv[1]  # Replace 'y' and 'x' with the coordinates of a representative yellow pixel
        print(f'HSV values: {hsv_pixel}')

    lower_yellow = np.array([80, 100, 200])  # Slightly lower than your representative yellow pixel
    upper_yellow = np.array([100, 255, 255])

    # Threshold the HSV image to get only yellow colors
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    if DEBUG:
        cv2.imwrite(f"debug_mask_{captcha_id}.png", mask)

    # Copy the original image
    res = bgr.copy()

    # Replace yellow outline with black in the copy of the original image
    res[mask > 0] = ([11, 11, 11, 255])  # Change color to black, alpha to 255
    if DEBUG:
        cv2.imwrite(f"debug_res_{captcha_id}.png", res)

    # Convert back to RGBA color space
    result = cv2.cvtColor(res, cv2.COLOR_BGRA2RGBA)
    if DEBUG:
        cv2.imwrite(f"debug_result_{captcha_id}.png", result)

    return result

def adjust_saturation_and_brightness(image, saturation_scale, brightness_scale):
    """
    Adjusts the saturation and brightness of an image.
    :param image: Input image.
    :param saturation_scale: Float indicating the saturation scaling factor.
    :param brightness_scale: Float indicating the brightness scaling factor.
    :return: Adjusted image.
    """
    if image.shape[2] == 3:  # Check if the image has 3 channels (RGB)
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        hsv[..., 1] = hsv[..., 1] * saturation_scale
        hsv[..., 2] = hsv[..., 2] * brightness_scale
        hsv[..., 1] = np.clip(hsv[..., 1], 0, 255)
        hsv[..., 2] = np.clip(hsv[..., 2], 0, 255)
        image = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    elif image.shape[2] == 4:  # Check if the image has 4 channels (RGBA)
        rgb = image[:, :, :3]
        rgba = cv2.cvtColor(rgb, cv2.COLOR_RGB2RGBA)
        rgba[..., 3] = image[..., 3]
        rgba[..., :3] = rgba[..., :3] * saturation_scale
        rgba[..., :3] = np.clip(rgba[..., :3] * brightness_scale, 0, 255)
        image = cv2.cvtColor(rgba, cv2.COLOR_RGBA2RGB)
    else:
        raise ValueError("Invalid image shape. Expected 3 or 4 channels.")

    return image
def process_image_for_edges(image):
    """
    Applies Canny Edge detection on a grayscale version of the image.
    :param image: Input image.
    :return: Grayscale image with edges highlighted.
    """
    # Convert to grayscale if image is not already grayscale
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    # Perform Canny edge detection
    edges = cv2.Canny(gray, 50, 150)

    
    if DEBUG:
        cv2.imwrite(f'edges_{captcha_id}.png', edges)
    return edges

def crop_puzzle_piece(captcha_id = "0cef46d889dc4b9b9970abedbd9a8a18.png"):
    """
    Detect and crop the puzzle piece from the small image, return the puzzle piece
    with the fixed size of (35x35).
    :param captcha_id: ID of the captcha image.
    :return: Cropped piece of the image.
    """
    # Load the image
    image = cv2.imread(f"small_imgs/{captcha_id}.png", cv2.IMREAD_UNCHANGED)

    

    # Set alpha values lower than 200 to 0
    image[image[:,:,3] < 200] = [0, 0, 0, 0]
    
    # Save the loaded image
    if DEBUG:
        cv2.imwrite(f'loaded_{captcha_id}.png', image)
    
    # Get the alpha channel
    alpha_channel = image[:,:,3]

    # Find the first non-zero row in alpha_channel, which corresponds to startY
    startY = np.where(alpha_channel > 200)[0][0]

    # Define the width and height of your piece
    width = 35
    height = 35

    # Now you can crop the image
    piece = image[startY:startY+height, 0:width]

    # Save the cropped image
    if DEBUG:
        cv2.imwrite(f'piece_{captcha_id}.png', piece)
    return piece



def match_piece_to_gap(cropped_piece, image):
    """
    Matches the cropped puzzle piece to a gap in a given image using template matching.
    :param cropped_piece: The cropped piece from the image.
    :param image: The full image where the piece needs to be placed.
    :return: Location of the best match of the piece in the image.
    """
    piece = cropped_piece
    
    # Darken the yellow outline of the puzzle piece
    piece = darken_yellow_outline(piece)
    
    if DEBUG:
        cv2.imwrite(f"piece_after_yellow_darkened_{captcha_id}.png", piece)

    # Adjust the saturation and brightness
    image = adjust_saturation_and_brightness(image, 3, 1.01)
    piece = adjust_saturation_and_brightness(piece, 1, 1)
    
    # Process the images for edge detection
    image_edges = process_image_for_edges(image)
    piece_edges = process_image_for_edges(piece)

    # The dimensions of the puzzle piece
    w = piece.shape[1]
    h = piece.shape[0]

    methods = ['cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR', 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
    colors = [(0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255), (255, 0, 255)]  # Green, Red, Yellow, Cyan, Magenta
    locations = []
    for method, color in zip(methods, colors):
        # Perform template matching
        result = cv2.matchTemplate(image, piece, eval(method))
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

        # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum, else take maximum
        match_loc = min_loc if method in ['cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED'] else max_loc

        # Draw a rectangle around the matched area
        cv2.rectangle(image, match_loc, (match_loc[0] + w, match_loc[1] + h), color, 2)
        locations.append(match_loc)

    # Display the image
    if SHOW_IMAGES:
        cv2.imshow('Detected', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return locations

def load_images(captcha_id):
    """
    Loads the original captcha image and the cropped puzzle piece image.
    :param captcha_id: ID of the captcha.
    :return: Original image and the cropped puzzle piece.
    """
    image = cv2.imread(f'big_imgs/{captcha_id}.jpg', cv2.IMREAD_UNCHANGED)
    cropped_piece = crop_puzzle_piece(captcha_id)
    return image, cropped_piece



In [ ]:

def solve_captcha(driver):
    while True:
        try:
            wait = WebDriverWait(driver, 10)
            iframe1 = wait.until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "iframe")
                )
            )
        except TimeoutException:
            print("iframe not found")
            return False

        driver.switch_to.frame(iframe1)
        print("convert iframe") 
       

        time.sleep(3)
        imgid=1234
        network_requests = driver.execute_script("return window.performance.getEntries()")
        if not os.path.exists('big_imgs'):
            os.makedirs('big_imgs')
        if not os.path.exists('small_imgs'):
            os.makedirs('small_imgs')
        for request in network_requests:
            if request["name"].endswith(".jpg"):
                print("save the background")
                response = requests.get(request["name"])
                with open(f'big_imgs/{imgid}.png', 'wb') as file:
                    file.write(response.content)
            elif request["name"].endswith(".frag.png"):
                print("save the slider")
                response = requests.get(request["name"])
                with open(f'small_imgs/{imgid}.png', 'wb') as file:
                    file.write(response.content)
                
                break
        image, puzzle_piece = load_images(imgid)
        locations = match_piece_to_gap(puzzle_piece, image)
        for XYCoordinates in locations:

            slidex = XYCoordinates[0]
            #element = driver.find_element_by_css_selector(".slider")
            try:
                # Find the slider element by class name
                element = driver.find_element(By.CLASS_NAME, "slider")
            except NoSuchElementException:
                print("Slider element not found")
                break

            # Get the initial position of the slider
            start_x = 0
            start_y = 0

            # Calculate the final position of the slider
            end_x = slidex
            end_y = 0

            # Simulate the drag and drop action
            action_chains = ActionChains(driver)
            action_chains.click_and_hold(element).move_by_offset(end_x - start_x, end_y - start_y).release().perform()

            # Use WebDriverWait to wait until the captcha is no longer present
            wait = WebDriverWait(driver, 20)
            wait.until_not(EC.presence_of_element_located((By.CSS_SELECTOR, "iframe")))

            # Break the loop if the captcha is no longer present
            if not EC.presence_of_element_located((By.CSS_SELECTOR, "iframe")):
                break

In [ ]:
import json
import time
import pickle
from numpy import random
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from seleniumrequests import Chrome
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.firefox.options import Options

options = webdriver.ChromeOptions() 
 
# Adding argument to disable the AutomationControlled flag 
options.add_argument("--disable-blink-features=AutomationControlled") 
 
# Exclude the collection of enable-automation switches 
options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
options.add_argument("--disable-blink-features=AutomationControlled")  # Disables Blink features used for automation detection
#options.add_argument("--disable-extensions")  # Disables browser extensions
options.add_argument("--disable-infobars")  # Disables the "Chrome is being controlled by automated test software" infobar
options.add_argument("--disable-notifications")  # Disables browser notifications
options.add_argument("--disable-popup-blocking")  # Disables popup blocking
options.add_argument("--disable-geolocation")  # Disables geolocation
options.add_argument("--allow-file-access-from-files")

# Instantiate the Chrome WebDriver with options

# Set custom user-agent string
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36") 
# Turn-off userAutomationExtension 
options.add_experimental_option("useAutomationExtension", False) 
 
# Setting the driver path and requesting a page 
driver = webdriver.Chrome(options=options) 
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {  #bypass the bot detection, preventing we are not using selenium
            "source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})""",
        })
"""if os.path.isfile("chrome_cookies.pkl"):
    with open('chrome_cookies.pkl', 'rb') as f:
        cookies = pickle.load(f)
    # Add cookies to the driver
    for cookie_dict in cookies:
        driver.add_cookie(cookie_dict)"""
# Changing the property of the navigator value for webdriver to undefined 
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
driver.get('https://onma.top/manga/noblesse')
"""with open("rondon_history.txt","r") as f:
    for line in reversed(f.readlines()):
        driver.execute_script("window.history.pushState({}, '', '"+line.strip().rstrip()+"')")
        time.sleep(random.uniform(2, 4))
        driver.forward()

driver.get('https://witanime.com/')
for num in range(1,1001,1):
        driver.execute_script("window.history.pushState({}, '', 'https://witanime.com/episode/one-piece-%d8%a7%d9%84%d8%ad%d9%84%d9%82%d8%a9-"+str(num)+"')")
        #time.sleep(random.uniform(2, 4))
        driver.forward()
        
for num in range(1,700,1):
        driver.execute_script("window.history.pushState({}, '', 'https://witanime.com/episode/naruto-shippuuden-%d8%a7%d9%84%d8%ad%d9%84%d9%82%d8%a9-"+str(num)+"')")
        #time.sleep(random.uniform(2, 4))
        driver.forward()"""
        
              
          
        
        
pickle.dump(driver.get_cookies(), open("chrome_cookies.pkl", "wb"))

url="https://auth.visas-fr.tlscontact.com/auth/realms/atlas/protocol/openid-connect/auth?response_type=code&client_id=web_app&scope=openid%20roles%20atlas%20web-origins%20email%20offline_access%20profile%20address%20phone&state=gHml9Y7uT7xBk56ZYGcryPeReiobuAVLxlKIOj-pqIQ%3D&redirect_uri=https://visa-fr.tlscontact.com/login/oauth2/code/oidc&issuer=maCAS2fr"
driver.get(url)
##############################
####username and password here 
##############################
username="hane***********"
passwd="#ySp***********"
##############################


#### login #######
#################
wait = WebDriverWait(driver, 10)
form = wait.until(EC.presence_of_element_located((By.ID, "kc-form-login")))

driver.find_element(By.ID, "username").send_keys(username)
driver.find_element(By.ID, "password").send_keys(passwd)
form.submit()


################################

################################
solve_captcha(driver)


In [1]:
pwd

'/home/bio/Documents/GitHub/python3'